<a href="https://colab.research.google.com/github/IvanKatorgin/Workshops-and-hackathons/blob/main/%D0%92%D1%8B%D0%B1%D0%BE%D1%80_%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Выбор оптимальной модели Whisper: анализ точности и скорости распознавания речи

Цель:

Сравнение производительности двух версий модели Whisper (Tiny vs Base) на датасете аудиозаписей по двум ключевым метрикам:
- Точность (Word Error Rate - WER)
- Скорость работы (время транскрипции)

### Dataset

https://huggingface.co/datasets/hf-internal-testing/librispeech_asr_dummy

### Install libs

In [1]:
!pip install openai-whisper jiwer datasets torchcodec librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=edf4c3c09d9e178539b35a83cf0308088b3348be38f1477434a1e543df925652
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


### Import required libs

In [2]:
import whisper
import time
import pandas as pd
import datasets
import librosa
import io
import soundfile as sf
import numpy as np

from jiwer import wer
from datasets import load_dataset, Audio
from tqdm import tqdm as tqdm

### Import required models

In [3]:
print("Loading Whisper models...")
model_tiny = whisper.load_model("tiny")
model_base = whisper.load_model("base")
print("Models loaded successfully!\n")

Loading Whisper models...


100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 110MiB/s]
100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 117MiB/s]


Models loaded successfully!



### Load a small audio dataset (LibriSpeech test-clean subset)

In [4]:
print("Loading dataset...")
dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

# Use first 100 samples for testing
dataset = dataset.select(range(min(100, len(dataset))))

# Disable torchcodec decoding
# means the dataset keeps audio as bytes, not decoded yet — faster for loading & manual resampling
dataset = dataset.cast_column("audio", Audio(decode=False))

print(f"Loaded {len(dataset)} audio samples\n")

Loading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

clean/validation-00000-of-00001.parquet:   0%|          | 0.00/9.19M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

Loaded 73 audio samples



In [5]:
# Reads raw audio bytes → waveform
# Ensures sampling rate = 16k, which Whisper requires
# Returns a clean float32 numpy array

def load_audio(sample):
    audio_bytes = sample["audio"]["bytes"]
    data, sr = sf.read(io.BytesIO(audio_bytes), dtype='float32')

    if sr != 16000:
        data = librosa.resample(data, orig_sr=sr, target_sr=16000).astype(np.float32)

    return data

### Function to transcribe and calculate metrics

In [6]:
def evaluate_model(model, model_name, dataset):
    predictions = []
    references = []
    times = []

    print(f"\nEvaluating {model_name}...")
    print("-" * 50)

    for i, sample in tqdm(enumerate(dataset), total=len(dataset)):

        reference = sample["text"].lower()

        # Load audio from raw bytes
        audio = load_audio(sample)

        start_time = time.time()
        result = model.transcribe(audio, fp16=False)
        inference_time = time.time() - start_time

        prediction = result["text"].lower().strip()

        predictions.append(prediction)
        references.append(reference)
        times.append(inference_time)

        print(f"Sample {i+1}/{len(dataset)} - Time: {inference_time:.2f}s")

    # Calculate WER
    wer_score = wer(references, predictions)
    avg_time = sum(times) / len(times)

    return {
        "model": model_name,
        "wer": wer_score,
        "avg_inference_time": avg_time,
        "predictions": predictions,
        "references": references
    }

### Evaluate both models

In [7]:
results_tiny = evaluate_model(model_tiny, "Whisper Tiny", dataset)
results_base = evaluate_model(model_base, "Whisper Base", dataset)


Evaluating Whisper Tiny...
--------------------------------------------------


  1%|▏         | 1/73 [00:02<02:38,  2.19s/it]

Sample 1/73 - Time: 2.18s


  3%|▎         | 2/73 [00:03<02:04,  1.76s/it]

Sample 2/73 - Time: 1.44s


...

Sample 71/73 - Time: 3.29s


 99%|█████████▊| 72/73 [04:08<00:03,  3.38s/it]

Sample 72/73 - Time: 3.44s


100%|██████████| 73/73 [04:12<00:00,  3.46s/it]

Sample 73/73 - Time: 3.52s


### Display comparison results

In [8]:
print("\n" + "="*60)
print("COMPARISON RESULTS")
print("="*60)

comparison_df = pd.DataFrame([
    {
        "Model": "Whisper Tiny",
        "WER": f"{results_tiny['wer']:.4f}",
        "Avg Inference Time (s)": f"{results_tiny['avg_inference_time']:.2f}"
    },
    {
        "Model": "Whisper Base",
        "WER": f"{results_base['wer']:.4f}",
        "Avg Inference Time (s)": f"{results_base['avg_inference_time']:.2f}"
    }
])
print(comparison_df.to_string(index=False))
print("\n")


COMPARISON RESULTS
       Model    WER Avg Inference Time (s)
Whisper Tiny 0.2400                   1.60
Whisper Base 0.2235                   3.45




### Show sample predictions

In [9]:
print("="*60)
print("SAMPLE PREDICTIONS")
print("="*60)

for i in range(min(3, len(dataset))):
    print(f"\nSample {i+1}:")
    print(f"Reference:    {results_tiny['references'][i]}")
    print(f"Tiny Model:   {results_tiny['predictions'][i]}")
    print(f"Base Model:   {results_base['predictions'][i]}")

SAMPLE PREDICTIONS

Sample 1:
Reference:    mister quilter is the apostle of the middle classes and we are glad to welcome his gospel
Tiny Model:   mr. quilter is the apostle of the middle classes and we are glad to welcome his gospel.
Base Model:   mr. quilter is the apostle of the middle classes, and we are glad to welcome his gospel.

Sample 2:
Reference:    nor is mister quilter's manner less interesting than his matter
Tiny Model:   nor is mr. quilters' manner less interesting than his matter.
Base Model:   nor is mr. quilter's manner less interesting than his matter.

Sample 3:
Reference:    he tells us that at this festive season of the year with christmas and roast beef looming before us similes drawn from eating and its results occur most readily to the mind
Tiny Model:   he tells us that at this festive season of the year, with christmas and roast beef looming before us, similarly drawn from eating and its results occur most readily to the mind.
Base Model:   he tells us that

### Find samples where models differ and at least one is wrong

In [10]:
interesting_samples = []
for i in range(len(dataset)):
    ref = results_tiny['references'][i]
    tiny_pred = results_tiny['predictions'][i]
    base_pred = results_base['predictions'][i]

    # Check if models differ from each other AND at least one differs from the reference
    models_differ = tiny_pred != base_pred
    at_least_one_wrong = (tiny_pred != ref) or (base_pred != ref)

    if models_differ and at_least_one_wrong:
        interesting_samples.append(i)

if interesting_samples:
    print(f"\nFound {len(interesting_samples)} samples where models disagree and differ from reference:\n")
    for idx in interesting_samples[:5]:  # Show up to 5 examples
        print(f"Sample {idx+1}:")
        print(f"Reference:    {results_tiny['references'][idx]}")
        print(f"Tiny Model:   {results_tiny['predictions'][idx]}")
        print(f"Base Model:   {results_base['predictions'][idx]}")
        print()
else:
    print("\nNo samples found where models disagree and differ from reference.")
    print("Showing first 3 samples instead:\n")
    for i in range(min(3, len(dataset))):
        print(f"Sample {i+1}:")
        print(f"Reference:    {results_tiny['references'][i]}")
        print(f"Tiny Model:   {results_tiny['predictions'][i]}")
        print(f"Base Model:   {results_base['predictions'][i]}")
        print()


Found 50 samples where models disagree and differ from reference:

Sample 1:
Reference:    mister quilter is the apostle of the middle classes and we are glad to welcome his gospel
Tiny Model:   mr. quilter is the apostle of the middle classes and we are glad to welcome his gospel.
Base Model:   mr. quilter is the apostle of the middle classes, and we are glad to welcome his gospel.

Sample 2:
Reference:    nor is mister quilter's manner less interesting than his matter
Tiny Model:   nor is mr. quilters' manner less interesting than his matter.
Base Model:   nor is mr. quilter's manner less interesting than his matter.

Sample 4:
Reference:    he has grave doubts whether sir frederick leighton's work is really greek after all and can discover in it but little of rocky ithaca
Tiny Model:   he has grave doubts whether sir frederick layton's work is really greek after all and can discover in it but little of rocky ithaca.
Base Model:   he has graved doubts whether sir frederick layton's 

### Summary

In [11]:
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
wer_diff = (results_base['wer'] - results_tiny['wer']) * 100
time_diff = results_base['avg_inference_time'] - results_tiny['avg_inference_time']

print(f"WER Base: {results_base['wer']*100:.2f}")
print(f"WER Tiny: {results_tiny['wer']*100:.2f}")
print(f"WER Difference: {wer_diff:.2f} percentage points (Base vs Tiny)")
print(f"Time Difference: {time_diff:.2f} seconds (Base vs Tiny)")
print(f"\nBase model is {'better' if results_base['wer'] < results_tiny['wer'] else 'worse'} in accuracy")
print(f"Base model is {'slower' if time_diff > 0 else 'faster'} in inference")


SUMMARY
WER Base: 22.35
WER Tiny: 24.00
WER Difference: -1.65 percentage points (Base vs Tiny)
Time Difference: 1.85 seconds (Base vs Tiny)

Base model is better in accuracy
Base model is slower in inference


Основные выводы:
1. Модель Base демонстрирует более высокую точность (ниже WER) по сравнению с Tiny
2. Модель Tiny значительно быстрее в обработке аудио
3. Улучшение точности достигается ценой увеличения времени обработки